# **Threads - Escalonamento e Sincronismo** 

### **Autoras** 👩‍💻

Desenvolvedores que contribuíram para a estruturação e desenvolvimento deste projeto:

- **Iza Lopes Ribeiro** - [Izalp](https://github.com/Izalp)
- **Wiliane Carolina Silva** - [wilicarol](https://github.com/wilicarol)

## **Simulação das modalidades**

In [1]:
import threading
import time
import random
from sports import sports


# Função para simular a execução do esporte
def sport(title, duration):
    start_time = time.perf_counter()  
    print(f'Modalidade {title:<30} | Início: {start_time:.3f}')

    time.sleep(duration)  # Simula a execução do esporte pelo tempo de duração

    end_time = time.perf_counter() 
    print(f'Modalidade {title:<30} | Término: {end_time:.3f} | Duração: {duration:.3f} horas')
    return duration

## **Escalonamento FCFS (First-Come, First-Served)**

**Descrição:** 
- Modalidades compartilham o mesmo Local de Evento;
- Modalidades são executadas na ordem de chegada, sem interrupções (a primeiro que chega é a primeira a usar o Local de Evento);
- O tempo de espera de uma modalidade é o tempo de execução das outras modalidades anteriores (se ela for a primeira, o tempo de espera é zero).

In [2]:
def escalonamento_fcfs(pending_sports):
    total_duration = 0  # Total de duração dos esportes
    total_wait_time = 0  # Total de tempo de espera
    previous_end_time = 0  # Tempo em que o último esporte terminou
    wait_times = []  # Lista para armazenar os tempos de espera de cada esporte

    for title, duration in pending_sports:
        wait_time = previous_end_time  # O tempo de espera é o tempo que o último esporte levou para terminar
        wait_times.append(wait_time)  # Armazena o tempo de espera para cada esporte
        total_wait_time += wait_time
        
        # Inicia o esporte e aguarda sua conclusão
        t = threading.Thread(target=sport, args=(title, duration))
        t.start()
        t.join()  # Executa o esporte até completar

        # Atualiza os totais
        total_duration += duration
        previous_end_time += duration  # Atualiza o tempo em que o próximo esporte começa a ser executado

    return total_duration, total_wait_time, len(pending_sports), wait_times  

In [3]:
def start_olympics_fcfs():
    print('\nOs Jogos Olímpicos com FCFS foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []         # Lista de esportes que não puderam ser executados

    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias para cada esporte
        pending_sports.append((title, duration))

    total_duration, total_wait_time, num_sports, wait_times = escalonamento_fcfs(pending_sports)

    # Calcula a média de duração e de tempo de espera
    avg_duration_time = total_duration / num_sports if num_sports else 0
    avg_wait_time = total_wait_time / num_sports if num_sports else 0
    
    print('\n')
    # Exibe os tempos de espera para cada esporte
    for i, (title, _) in enumerate(pending_sports):
        print(f'Modalidade {title:<25} | Tempo de Espera: {wait_times[i]:.3f} horas')

    print(f'\nMédia de duração dos esportes: {avg_duration_time:.3f} horas')
    print(f'Média de tempo de espera (FCFS): {avg_wait_time:.3f} horas\n')

start_olympics_fcfs()


Os Jogos Olímpicos com FCFS foram iniciados!

Modalidade Ciclismo Bmx Racing            | Início: 2379289.463
Modalidade Ciclismo Bmx Racing            | Término: 2379293.280 | Duração: 3.816 horas
Modalidade Judô                           | Início: 2379293.282
Modalidade Judô                           | Término: 2379296.135 | Duração: 2.852 horas
Modalidade Canoagem Slalom                | Início: 2379296.139
Modalidade Canoagem Slalom                | Término: 2379299.970 | Duração: 3.830 horas
Modalidade Nado Artístico                 | Início: 2379299.976
Modalidade Nado Artístico                 | Término: 2379300.977 | Duração: 1.001 horas
Modalidade Ciclismo de Pista              | Início: 2379300.985
Modalidade Ciclismo de Pista              | Término: 2379302.341 | Duração: 1.355 horas
Modalidade Remo                           | Início: 2379302.348
Modalidade Remo                           | Término: 2379303.857 | Duração: 1.508 horas
Modalidade Taekwondo                     

## **Escalonamento Round Robin (RR)**

**Descrição:**
- Cada modalidade recebe um tempo fixo (quantum) para utilizar o Local de Evento. Se ela não terminar até o tempo limite, volta ao final da fila;
- A duração das olimpíadas (performance) depende do tempo limite (quantum) que as modalidades tem para utilizar o Local do Evento.

In [10]:
def escalonamento_rr(pending_sports, lock, completed_sports, quantum=2):
    current_time = time.perf_counter()  # Momento inicial
    wait_times = []

    while pending_sports:
        for i in range(len(pending_sports)):
            title, duration, arrival_time = pending_sports.pop(0)
            effective_duration = min(duration, quantum)
            lock.acquire()  # Protege a seção crítica
            
            # Calcular tempo de espera antes da execução do esporte
            wait_time = current_time - arrival_time
            wait_times.append(wait_time)
            
            # Cria e executa o thread para o esporte com duração limitada pelo quantum=2
            t = threading.Thread(target=sport, args=(title, effective_duration))
            t.start()
            t.join()
            lock.release()  # Libera a seção crítica

            # Atualiza o tempo atual após a execução
            current_time = time.perf_counter()

            # Se o esporte não terminou, retorna com o tempo restante
            if duration > quantum:
                pending_sports.append((title, duration - quantum, current_time))
            else:
                completed_sports.append((title, wait_time))
    
    return wait_times

In [11]:
def start_olympics_rr():
    print('\nOs Jogos Olímpicos com RR foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []         # Lista de esportes que não puderam ser executados
    lock = threading.Lock()     # Bloqueio para que apenas um esporte acesse o Local de Evento por vez
    completed_sports = []       # Lista de esportes concluídos e seus tempos de espera
    total_durations = []        # Lista para armazenar os tempos de duração

    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias de cada esporte
        arrival_time = time.perf_counter()    # Tempo de chegada na fila

        pending_sports.append((title, duration, arrival_time))
        total_durations.append(duration)  # Adiciona a duração ao total

    wait_times = escalonamento_rr(pending_sports, lock, completed_sports)

    print('\n')
    # Exibe os tempos de espera para cada esporte
    for i, (title, _) in enumerate(completed_sports):
        print(f'Modalidade {title:<25} | Tempo de Espera: {wait_times[i]:.3f} horas')

    # Calcula a média dos tempos de duração
    avg_duration = sum(total_durations) / len(total_durations) if total_durations else 0
    print(f'\nMédia de duração dos esportes: {avg_duration:.3f} horas')

    # Calcula a média dos tempos de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    print(f'Média de tempo de espera (RR): {avg_wait_time:.3f} horas\n')


# Inicia a simulação com Round Robin
start_olympics_rr()


Os Jogos Olímpicos com RR foram iniciados!

Modalidade Escalada                       | Início: 2382871.498
Modalidade Escalada                       | Término: 2382873.499 | Duração: 2.000 horas
Modalidade Atletismo                      | Início: 2382873.507
Modalidade Atletismo                      | Término: 2382875.508 | Duração: 2.000 horas
Modalidade Saltos Ornamentais             | Início: 2382875.511
Modalidade Saltos Ornamentais             | Término: 2382877.511 | Duração: 2.000 horas
Modalidade Luta                           | Início: 2382877.512
Modalidade Luta                           | Término: 2382878.051 | Duração: 0.538 horas
Modalidade Maratona Aquática              | Início: 2382878.057
Modalidade Maratona Aquática              | Término: 2382880.058 | Duração: 2.000 horas
Modalidade Ciclismo de Estrada            | Início: 2382880.065
Modalidade Ciclismo de Estrada            | Término: 2382882.066 | Duração: 2.000 horas
Modalidade Tênis                          |

## **Implementação dos Mecanismos de Sincronização**

###  **Semáforo Binário**

**Descrição:** 

- O threading.Semaphore age como uma **contagem de permissões**, controlando quantas modalidades podem acessar o Local de Evento ao mesmo tempo;

- Para ter o comportamento de um semáforo binário, foi necessário inicializar com o valor 1;

- Controla o acesso de uma modalidade por vez para utilizar o Local de Evento (seção crítica), funcionando como um “lock” com valores 0 e 1.

In [12]:
binary_semaphore = threading.Semaphore(1)

def sport_with_binary_semaphore(title, duration):
    wait_start = time.perf_counter()
    print(f'Modalidade {title:<30} | Aguardando acesso ao Local de Evento')
    with binary_semaphore:
        wait_time = time.perf_counter() - wait_start
        print(f'Modalidade {title:<30} | Tempo de espera: {wait_time:.3f} horas')
        print(f'Modalidade {title:<30} | Iniciou a execução')
        sport(title, duration)
    print(f'Modalidade {title:<30} | Liberou acesso ao Local de Evento\n{"-"*80}')

In [13]:
def run_sports(sports, method):
    print('\nOs Jogos Olímpicos com Semáforo Binário foram iniciados!\n')
    threads = []
    start_time = time.perf_counter()
    
    for title in sports:
        duration = random.uniform((1/60),5)
        thread = threading.Thread(target=method, args=(title, duration))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
    
    end_time = time.perf_counter()
    return end_time - start_time


binary_semaphore_time = run_sports(sports, sport_with_binary_semaphore)

print(f'\n[Semáforo Binário] Tempo total: {binary_semaphore_time:.3f} horas\n')


Os Jogos Olímpicos com Semáforo Binário foram iniciados!

Modalidade Atletismo                      | Aguardando acesso ao Local de Evento
Modalidade Atletismo                      | Tempo de espera: 0.000 horas
Modalidade Atletismo                      | Iniciou a execução
Modalidade Atletismo                      | Início: 2383637.996
Modalidade Badminton                      | Aguardando acesso ao Local de Evento
Modalidade Basquete                       | Aguardando acesso ao Local de Evento
Modalidade Basquete 3x3                   | Aguardando acesso ao Local de Evento
Modalidade Boxe                           | Aguardando acesso ao Local de Evento
Modalidade Breaking                       | Aguardando acesso ao Local de Evento
Modalidade Canoagem de Velocidade         | Aguardando acesso ao Local de Evento
Modalidade Canoagem Slalom                | Aguardando acesso ao Local de Evento
Modalidade Ciclismo Bmx Freestyle         | Aguardando acesso ao Local de Evento
Modalidade C

### **Monitor**

**Descrição:** 
- Utiliza uma Condition para controlar o acesso de só uma modalidade por vez Local de Evento (exclusão mútua);

- **Variáveis de Condição:** monitor_condition.wait() e monitor_condition.notify_all() implementam uma fila de espera organizada, onde as modalidades aguardam enquanto o local está ocupado e são notificadas assim que ele é liberado;

- **Comportamento com Condition:** Cada modalidade tenta acessar o Local de Evento. Se ele estiver ocupado (local_ocupado = True), a modalidade aguarda usando wait(). Após o término de uma modalidade, notify_all() notifica as modalidades em espera, permitindo que a próxima modalidade acesse o local.

In [20]:
monitor_condition = threading.Condition()
local_ocupado = False

def sport_with_monitor(title, duration):
    wait_start = time.perf_counter()
    print(f'Modalidade {title:<30} | Aguardando acesso ao Local de Evento')

    with monitor_condition:
        global local_ocupado  
        
        # Aguarda até que o local esteja disponível
        while local_ocupado:
            monitor_condition.wait()
        
        # Indica que o local agora está ocupado
        local_ocupado = True

        # Calcula o tempo de espera e inicia a execução
        wait_time = time.perf_counter() - wait_start
        print(f'Modalidade {title:<30} | Tempo de espera: {wait_time:.3f} horas')
        print(f'Modalidade {title:<30} | Iniciou a execução')
        
        # Executa o esporte
        sport(title, duration)

        # Libera o local e notifica outras threads
        local_ocupado = False
        monitor_condition.notify_all()

    print(f'Modalidade {title:<30} | Liberou acesso ao Local de Evento\n{"-"*80}')

In [21]:
def run_sports(sports, method):
    print('\nOs Jogos Olímpicos com Monitor foram iniciados!\n')
    threads = []
    start_time = time.perf_counter()
    
    for title in sports:
        duration = random.uniform((1/60),5)  
        thread = threading.Thread(target=method, args=(title, duration))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
    
    end_time = time.perf_counter()
    return end_time - start_time

monitor_time = run_sports(sports, sport_with_monitor)
print(f'\n[Monitor] Tempo total: {monitor_time:.3f} horas\n')


Os Jogos Olímpicos com Monitor foram iniciados!

Modalidade Atletismo                      | Aguardando acesso ao Local de Evento
Modalidade Atletismo                      | Tempo de espera: 0.000 horas
Modalidade Atletismo                      | Iniciou a execução
Modalidade Atletismo                      | Início: 2385314.042
Modalidade Badminton                      | Aguardando acesso ao Local de Evento
Modalidade Basquete                       | Aguardando acesso ao Local de Evento
Modalidade Basquete 3x3                   | Aguardando acesso ao Local de Evento
Modalidade Boxe                           | Aguardando acesso ao Local de Evento
Modalidade Breaking                       | Aguardando acesso ao Local de Evento
Modalidade Canoagem de Velocidade         | Aguardando acesso ao Local de Evento
Modalidade Canoagem Slalom                | Aguardando acesso ao Local de Evento
Modalidade Ciclismo Bmx Freestyle         | Aguardando acesso ao Local de Evento
Modalidade Ciclismo B

### **Semáforo de Contagem**

**Descrição:** 
- Permite acesso simultâneo de várias modalidades ao Local de Evento (seção crítica) até um limite definido pela comissão (contador);

- **Contém uma restrição adicional:** ele impede que o semáforo seja liberado mais vezes do que foi adquirido (evitando liberações extras).

In [15]:
counting_semaphore = threading.BoundedSemaphore(2)  # Permite até 2 modalidades simultâneas

def sport_with_counting_semaphore(title, duration):
    wait_start = time.perf_counter()
    print(f'Modalidade {title:<30} | Aguardando acesso ao Local de Evento')
    with counting_semaphore:
        wait_time = time.perf_counter() - wait_start
        print(f'Modalidade {title:<30} | Tempo de espera: {wait_time:.3f} horas')
        print(f'Modalidade {title:<30} | Iniciou a execução')
        sport(title, duration)
    print(f'Modalidade {title:<30} | Liberou acesso ao Local de Evento\n{"-"*80}')

In [58]:
def run_sports(sports, method):
    print('\nOs Jogos Olímpicos com Semáforo de Contagem foram iniciados!\n')
    threads = []
    start_time = time.perf_counter()
    
    for title in sports:
        duration = random.uniform((1/60),5) 
        thread = threading.Thread(target=method, args=(title, duration))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
    
    end_time = time.perf_counter()
    return end_time - start_time


counting_semaphore_time = run_sports(sports, sport_with_counting_semaphore)
print(f'\n[Semáforo de Contagem] Tempo total: {counting_semaphore_time:.3f} horas\n')


Os Jogos Olímpicos com Semáforo de Contagem foram iniciados!

Modalidade Atletismo                      | Aguardando acesso ao Local de Evento
Modalidade Atletismo                      | Tempo de espera: 0.000 horas
Modalidade Atletismo                      | Iniciou a execução
Modalidade Atletismo                      | Início: 2348199.006
Modalidade Badminton                      | Aguardando acesso ao Local de Evento
Modalidade Badminton                      | Tempo de espera: 0.000 horas
Modalidade Badminton                      | Iniciou a execução
Modalidade Badminton                      | Início: 2348199.007
Modalidade Basquete                       | Aguardando acesso ao Local de Evento
Modalidade Basquete 3x3                   | Aguardando acesso ao Local de Evento
Modalidade Boxe                           | Aguardando acesso ao Local de Evento
Modalidade Breaking                       | Aguardando acesso ao Local de Evento
Modalidade Canoagem de Velocidade         | Aguarda

## **Escalonamento com Sincronismo**

### FCFS + Semáforo Binário

In [65]:

def fcfs_with_binary_semaphore(pending_sports, wait_times, duration_times):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        
        # Inicia a contagem do tempo de espera
        wait_start_time = time.perf_counter()  
        
        t = threading.Thread(target=sport_with_binary_semaphore, args=(title, duration))
        executed_today.append((t, title))
        t.start()
        t.join()  
        
        # Após a execução, calcula o tempo de espera
        wait_end_time = time.perf_counter()
        wait_time = wait_end_time - wait_start_time  # O tempo que a thread esperou
        wait_times.append(wait_time)  # Adiciona o tempo de espera à lista
        duration_times.append(duration)  # Adiciona a duração à lista

    return executed_today

In [66]:
def start_fcfs_binary_semaphore():
    print('\nOs Jogos Olímpicos com FCFS + Semáforo Binário foram iniciados!\n')
    all_sports = sports.copy()  # Cópia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados
    wait_times = []  # Lista para armazenar os tempos de espera
    duration_times = []  # Lista para armazenar as durações dos esportes

    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias em horas
        pending_sports.append((title, duration))

    fcfs_with_binary_semaphore(pending_sports, wait_times, duration_times)

    # Calcula a média do tempo de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    # Calcula a média do tempo de duração
    avg_duration_time = sum(duration_times) / len(duration_times) if duration_times else 0
    
    print(f'\nMédia de tempo de espera (FCFS + Semáforo Binário): {avg_wait_time:.3f} horas')
    print(f'Média de tempo de duração (FCFS + Semáforo Binário): {avg_duration_time:.3f} horas\n')

# Inicia a simulação com FCFS e semáforo binário
start_fcfs_binary_semaphore()


Os Jogos Olímpicos com FCFS + Semáforo Binário foram iniciados!

Modalidade Basquete 3x3                   | Aguardando acesso ao Local de Evento
Modalidade Basquete 3x3                   | Tempo de espera: 0.000 horas
Modalidade Basquete 3x3                   | Iniciou a execução
Modalidade Basquete 3x3                   | Início: 2350778.269
Modalidade Basquete 3x3                   | Término: 2350779.012 | Duração: 0.742 horas
Modalidade Basquete 3x3                   | Liberou acesso ao Local de Evento
--------------------------------------------------------------------------------
Modalidade Ciclismo de Estrada            | Aguardando acesso ao Local de Evento
Modalidade Ciclismo de Estrada            | Tempo de espera: 0.000 horas
Modalidade Ciclismo de Estrada            | Iniciou a execução
Modalidade Ciclismo de Estrada            | Início: 2350779.016
Modalidade Ciclismo de Estrada            | Término: 2350783.160 | Duração: 4.144 horas
Modalidade Ciclismo de Estrada       

### RR + Semáforo de Contagem

In [4]:
# Escalonamento RR com semáforo de contagem
def rr_with_counting_semaphore(pending_sports, wait_times, duration_times, quantum=2):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)

        t = threading.Thread(target=sport_with_counting_semaphore, args=(title, min(duration, quantum)))
        executed_today.append((t, title))
        t.start()
        t.join()

        # Se o esporte não terminou, retorna com o tempo restante
        if duration > quantum:
            pending_sports.append((title, duration - quantum))

        # Após a execução, calcula o tempo de espera
        wait_time = time.perf_counter()
        wait_times.append(wait_time)  # Adiciona o tempo de espera à lista
        duration_times.append(duration)  # Adiciona a duração à lista

    return executed_today

In [5]:
def start_rr_counting_semaphore():
    print('\nOs Jogos Olímpicos com RR foram iniciados!\n')
    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados no dia anterior
    wait_times = []  # Lista para armazenar os tempos de espera
    duration_times = []  # Lista para armazenar as durações dos esportes

    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias em horas
        pending_sports.append((title, duration))

    rr_with_counting_semaphore(pending_sports, wait_times, duration_times)

     # Calcula a média do tempo de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    # Calcula a média do tempo de duração
    avg_duration_time = sum(duration_times) / len(duration_times) if duration_times else 0
    
    print(f'\nMédia de tempo de espera (RR + Semáforo de contagem): {avg_wait_time:.3f} horas')
    print(f'Média de tempo de duração (RR + Semáforo de contagem): {avg_duration_time:.3f} horas\n')

# Inicia a simulação com RR e semáforo de contagem
start_rr_counting_semaphore()



Os Jogos Olímpicos com RR foram iniciados!

Modalidade Luta                           | Aguardando acesso ao Local de Evento
Modalidade Luta                           | Tempo de espera: 0.000 horas
Modalidade Luta                           | Iniciou a execução
Modalidade Luta                           | Início: 2352741.221
Modalidade Luta                           | Término: 2352743.012 | Duração: 1.791 horas
Modalidade Luta                           | Liberou acesso ao Local de Evento
--------------------------------------------------------------------------------
Modalidade Ciclismo Bmx Freestyle         | Aguardando acesso ao Local de Evento
Modalidade Ciclismo Bmx Freestyle         | Tempo de espera: 0.000 horas
Modalidade Ciclismo Bmx Freestyle         | Iniciou a execução
Modalidade Ciclismo Bmx Freestyle         | Início: 2352743.015
Modalidade Ciclismo Bmx Freestyle         | Término: 2352745.017 | Duração: 2.000 horas
Modalidade Ciclismo Bmx Freestyle         | Liberou acesso

### **Comparação de resultado:**

- FCFS será mais simples, com as modalidades executando uma por vez. O tempo total de execução será maior, pois nenhuma modalidade é interrompida.

- Round Robin com semáforo de contagem permite execuções simultâneas, mas interrompe modalidades que ultrapassam o quantum, o que pode melhorar a eficiência em termos de utilização de recursos.

**OBS.:** O monitor não foi utilizado diretamente no código anterior porque, na prática, ele é bastante semelhante ao semáforo binário.